Import useful modules 

In [ ]:
import sys 
import os 
### discomment above two line ones to add the feelpp path with nirb module in the python environement  
# feelpath = os.path.join('/data/home/elarif/devel/feelpp/build/python/install/lib/python3/dist-packages')
# sys.path.append(feelpath)
from feelpp.mor.nirb.nirb import *
from feelpp.mor.nirb.utils import init_feelpp_environment
from feelpp.mor.nirb.nirb_perf import ComputeErrorSampling, ComputeErrors
import time
import json 
import pandas as pd

Choose config file

In [ ]:
config_file =os.path.join("/data/home/elarif/devel/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.cfg")
N = None # number of basis function saved 
outdir = None 

Set feelpp environnement 

In [ ]:
cfg = feelpp.readCfg(config_file)
toolboxType = cfg['nirb']['toolboxType']
e = init_feelpp_environment(toolboxType, config_file)

nirb_file = feelpp.Environment.expand(cfg['nirb']['filename'])
config_nirb = feelpp.readJson(nirb_file)['nirb']

Get number of basis functions saved. By defaults, it will take the value filled in the model path. 

In [ ]:
nbSnap=N
if nbSnap==None:
    nbSnap = config_nirb['nbSnapshots']

Set the input/output path 

In [ ]:
doGreedy = config_nirb['greedy-generation']
doRectification = config_nirb['doRectification']
rectPath = ["noRect", "Rect"][doRectification]
greedyPath = ["noGreedy", "Greedy"][doGreedy]

if outdir is None:
    RESPATH = f"results/{rectPath}/{greedyPath}"
else:
    RESPATH = outdir

Initialize nirb offline class 

In [ ]:
nirb_on = nirbOnline(**config_nirb)

Load offline data

In [ ]:

start= time()

err = nirb_on.loadData(path=RESPATH, nbSnap=nbSnap)
assert err == 0, "Error while loading data"

Choose the parameter to cumpute nirb approximate solution 

In [ ]:

mu = nirb_on.Dmu.mumin() # get the min value of parameters given in the model


Compute approximate solution 

In [ ]:
uHh = nirb_on.getOnlineSol(mu)

finish = time()

Show infos 

In [ ]:

if feelpp.Environment.isMasterRank():
    print(f"[NIRB] Online Elapsed time = ", finish-start)
    print(f"[NIRB] Online part Done !")

# NIRB perfs 

We can compare the above nirb solution with the FE one. 

In [ ]:
errorN = ComputeErrors(nirb_on, mu, h1=True)
df = pd.DataFrame(errorN)
df['N'] = nirb_on.N

Display errors 

In [ ]:

if feelpp.Environment.isMasterRank():
    print(f"[NIRB online] with {nirb_on.N} basis functions ")
    print(f"[NIRB online] Convergence error between nirb and FE solution ")
    print(df.head())

Or get error on a sampling of parameters. This will compare the nirb solution with the FE one on the fine mesh computed from parameter in the sampling (take little time). 

In [ ]:
Nsample = 30
errorN = ComputeErrorSampling(nirb_on, Nsample=Nsample, h1=True)
df = pd.DataFrame(errorN)
df['N'] = nirb_on.N

Display errors 

In [ ]:

if feelpp.Environment.isMasterRank():
    print(f"[NIRB online] with {nirb_on.N} basis functions ")
    print(f"[NIRB online] Error on a sampling of {df.shape[0]} parameters ")
    data_mean = df.mean(axis=0)
    print("[NIRB online] Mean of errors ")
    print(data_mean)
    data_min = df.min(axis=0)
    print("[NIRB online] Min of errors ")
    print(data_min)
    data_max = df.max(axis=0)
    print("[NIRB online] Max of errors ")
    print(data_max)

Save covergence error values in a `csv` file for making statistics or vizualisation   

In [ ]:

file =RESPATH +f"/errors{Nsample}Params.csv"
header = not os.path.isfile(file)   
df.to_csv(file, mode='a', index=False, header=header)